In [2]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")

df_pitches = pd.read_csv("C:\\MLB_1.csv")
df_salaries = pd.read_csv("C:\\MLB_2.csv")
# Creating my merged data frame
data = df_pitches.merge(df_salaries, 
                        left_on='PLAYER', 
                        right_on='PLAYER', 
                        how='outer', 
                        suffixes=["","_SALARY"] #since they are the same name, we need a suffix
                       )

# Only selecing pitchers with a lot of pitches
#data = data[data.total_count > 1000][['PLAYER','SALARY']]

#Selecing people with missing salaries
no_missing_salaries = data[data.SALARY.notnull()]

#Displaying results
no_missing_salaries.reset_index(inplace=True,drop='index')
no_missing_salaries

# pip install fuzzywuzzy
from fuzzywuzzy import fuzz

def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in no_missing_salaries.PLAYER:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, df_salaries.PLAYER, 75)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"player_name" : name})
    dict_.update({"match_name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table


,player_name,match_name,score
0,Gregg Zau,Gregg Zaun,95
1,Henry Banco,Henry Blanco,96
2,Moises Alon,Moises Alou,91
3,Corey Paterson,Corey Patterson,97
4,Rod Barjas,Rod Barajas,95
...,...,...,...
332,Matt Treanor,Matt Treanor,100
333,Aaron Hill,Aaron Hill,100
334,Nick Markakis,Nick Markakis,100
335,Carlos Quentin,Carlos Quentin,100


In [4]:

# Data:
# https://www.kaggle.com/pschale/mlb-pitch-data-20152018?select=player_names.csv
